In [1]:
import xarray as xr
from netCDF4 import Dataset
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import load_data
import wrf

In [2]:
refl_yoffset = 256
refl_xoffset = 185

year = str(2012)

# W, QVAPOR, U, V
field = 'QVAPOR'

In [3]:
#
# read in the W data
#

input_W_data_path = '/glade/scratch/hardt/step/wrf3d'
output_data_path = '/glade/work/hardt/data/model2/v1'

#
# for testing only get the one file.
#
#W_ds = xr.open_mfdataset(os.path.join(input_W_data_path, year, "wrf3d_d01_2014-09-30_*"), combine="nested", concat_dim='Time')

#
# Read in all the data in the year directory.
#
W_ds = xr.open_mfdataset(os.path.join(input_W_data_path, year, "wrf3d_d01*"), combine="nested", concat_dim='Time')


In [4]:
W_yoffset = refl_yoffset + 130
W_xoffset = refl_xoffset + 560

print('W_yoffset:',W_yoffset)
print('W_xoffset:',W_xoffset)


W_yoffset: 386
W_xoffset: 745


In [5]:

W_t = W_ds.XTIME.values

#
# start values determined in refl_ds.attrs history
# ncks -O -dwest_east,560,1320 -dsouth_north,130,955
#
print('Loading', field, 'data.')

#
# get 3D data
#
# W - vertical wind
# QVAPOR - Water vapor
# U - U component of Wind
# V - V component of Wind
#
if field == 'W':
    VAR = W_ds.W[:,:,W_yoffset:W_yoffset+256,W_xoffset:W_xoffset+256].values
elif field =='QVAPOR':
    VAR = W_ds.QVAPOR[:,:,W_yoffset:W_yoffset+256,W_xoffset:W_xoffset+256].values
elif field =='U':
    VAR = W_ds.U[:,:,W_yoffset:W_yoffset+256,W_xoffset:W_xoffset+256].values
elif field =='V':
    VAR = W_ds.V[:,:,W_yoffset:W_yoffset+256,W_xoffset:W_xoffset+256].values
else:
    print("Field", field, "not an option.")
    
#
# Use this to get composite data
#
#W = W_ds.W[:,:,W_yoffset:W_yoffset+256,W_xoffset:W_xoffset+256].values.max(axis=1)

print(VAR.shape)
print('np.amin(VAR):',np.amin(VAR))
print('np.percentile(VAR, 99.9):',np.percentile(VAR, 99.9))


Loading QVAPOR data.
(1224, 50, 256, 256)
np.amin(VAR): 0.0
np.percentile(VAR, 99.9): 0.019267503172159195


In [6]:

output_data = os.path.join(output_data_path, "model2-v1_" + year + '_3D_' + field + '.nc')

W_OUT = xr.DataArray(VAR, name=field)
encoding={field: {'zlib': True, '_FillValue': -99.0}}

W_XTIME = xr.DataArray(W_t, name='XTIME')

print("Writing", field, "data to", output_data)
W_OUT.to_netcdf(output_data, encoding=encoding)
W_XTIME.to_netcdf(output_data, mode='a')

del VAR
del W_OUT
del W_ds

print("Done")

Writing QVAPOR data to /glade/work/hardt/data/model2/v1/model2-v1_2012_3D_QVAPOR.nc
Done
